# Fit 2-6 Class GMMs and Calculate Average Profiles

This notebook will fit 8 class GMMs to UK-ESM historical Southern Ocean data 2001-2018, following Jones et al. 2019 (https://doi.org/10.1029/2018JC014629). These files are required to reproduce Figures YY from *Heuristic Methods for Determining the Number of Classes in Unsupervised Classification of Climate Models*, E. Boland et al. 2022 (doi to follow). This requires cluster_utils.py and input datafiles via the googleapi CMIP6 store (see cluster_utils.py for more info)

Outputs stored in model/\[ensemble\]/\[nclasses\]/gmm.obj and data/\[ensemble\]/\[nclasses\]/avg.obj

Please attribute any plots or code from this notebook using the DOI from Zenodo: to come

Updated Nov 2022
E Atkinson & E Boland [emmomp@bas.ac.uk](email:emmomp@bas.ac.uk)

In [3]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:32988")
client

<Client: 'tcp://127.0.0.1:32988' processes=8 threads=8, memory=64.00 GiB>

In [1]:
from dask_gateway import Gateway
gateway = Gateway()
from dask.distributed import Client

'''properly shutdown any previous clusters'''
clusters=gateway.list_clusters()
if clusters != []:
    print(f'found {len(clusters)} clusters')
    for cluster in clusters:
        cluster = gateway.connect(cluster.name)
        client=Client(cluster)
        client.close()
        cluster.shutdown()

ValueError: No dask-gateway address provided or found in configuration

In [85]:
client.close()

In [8]:
from dask_gateway import Gateway
gateway = Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")
from dask.distributed import Client

'''properly shutdown any previous clusters'''
clusters=gateway.list_clusters()
if clusters != []:
    print(f'found {len(clusters)} clusters')
    for cluster in clusters:
        cluster = gateway.connect(cluster.name)
        client=Client(cluster)
        client.close()
        cluster.shutdown()

In [9]:
import dask_gateway

# Create a connection to dask-gateway.
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

# Inspect and change the options if required before creating your cluster.
options = gw.cluster_options()
options.worker_cores = 4

# Create a dask cluster, or, if one already exists, connect to it.
# This stage creates the scheduler job in SLURM, so may take some time.
# While your job queues.
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(options, shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

# Create at least one worker, and allow your cluster to scale to three.
cluster.adapt(minimum=1, maximum=3)

# Get a dask client.
client = cluster.get_client()

/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/distributed/client.py:1148: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+---------+
| Package     | client    | scheduler | workers |
+-------------+-----------+-----------+---------+
| cloudpickle | 1.6.0     | 2.1.0     | None    |
| dask        | 2021.05.0 | 2022.7.0  | None    |
| distributed | 2021.05.0 | 2022.7.0  | None    |
| lz4         | None      | 4.0.0     | None    |
| msgpack     | 1.0.2     | 1.0.4     | None    |
| toolz       | 0.11.1    | 0.12.0    | None    |
+-------------+-----------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [10]:
import numpy as np
import xarray as xr

import os
import pickle

import cluster_utils as flt

In [15]:
import importlib
importlib.reload(flt)

<module 'cluster_utils' from '/home/users/eboland/bas-pangeo-backup/emma-volume-backup2/emma-volume-backup/OceanClustering_paper/cluster_utils.py'>

### User options
Leave as is to recreate the paper

In [11]:
# Number of classes 
model='model_20012017'
classes = [2,3,4,5,6]
#Time range
tslice=slice('2001-01', '2017-12') 
#Depth range
levSel=slice(5, 2000)
ids = ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 'r4i1p1f2', 'r5i1p1f3', 'r6i1p1f3', 'r7i1p1f3', 'r8i1p1f2', 'r9i1p1f2', 'r10i1p1f2'] 
npca=3 #number of PCA components
ntrain=3000 #number of profiles per month to use in training dataset

Uncomment the following two lines if you need to generate mask.npy:

In [12]:
#data = flt.retrieve_profiles(timeRange = slice('1995-01', '1995-02'))
#np.save('data/mask', data['n'])
mask = np.load('data/mask_3000.npy', allow_pickle=True)

### Train models and generate average profiles for chosen ensemble members and classes 

In [ ]:
avg_profiles = {}
for m_id in ids:
    
    #Check if data already exists
    tests=[]
    for nn,n_classes in enumerate(classes):   
        path_n = '{}/{}/{}'.format(model,m_id, n_classes)
        tests.append(os.path.isfile('{}/probs.nc'.format(path_n)))
    if np.all(tests):
        print('Found classifications for {}, skipping'.format(m_id))
        continue
    else:
        print('Starting {}'.format(m_id))
        options = {'memberId' : m_id}
        path_id = '{}/{}'.format(model,m_id)
        # Check if models already created
        tests=[]
        for nn,n_classes in enumerate(classes):   
            path_data = '{}/{}/{}'.format(model,m_id, n_classes)
            tests.append(os.path.isfile('{}/gmm.obj'.format(path_data)))
        if np.all(tests): # All models trained, no need to load training set       
            with open('{}/pca.obj'.format(path_id),'rb') as file:
                pca=pickle.load(file)         
        else:     # Load training set, generate PCA model
            print('No models found, generating training set')
            [data_train,pca] = flt.generate_trainingset(timeRange = tslice, mask=mask, options=options,n_components=npca,N=ntrain,levSel=levSel)
            if not os.path.exists(path_id):
                os.makedirs(path_id)
            with open('{}/pca.obj'.format(path_id), 'wb') as file:
                pickle.dump(pca, file)  
        
        # Retrieve all Southern Ocean data
        options = {'memberId' : m_id}
        data = flt.retrieve_profiles(timeRange=tslice,mask=mask,options=options,levSel=levSel)
        data = data.chunk({'time': data.sizes['time'], 'n': 1024})
        # Normalise the samples
        data_norm = flt.normalise_data(data, ('n', 'time')) 
        # Transform to PCA space
        data_trans = flt.pca_transform(data_norm, pca)
        print('Finished setup for {}'.format(m_id))      

        for nn,n_classes in enumerate(classes):  
            path_n = '{}/{}/{}'.format(model,m_id, n_classes)
            #Check if model already created
            if os.path.isfile('{}/gmm.obj'.format(path_n)):
                with open('{}/gmm.obj'.format(path_n),'rb') as file:
                    gmm=pickle.load(file)                
            else:            
                print('Training {} class model'.format(n_classes))
                if not os.path.exists(path_n):
                    os.makedirs(path_n)            
                gmm = flt.train_gmm(data_train, n_classes)
                with open('{}/gmm.obj'.format(path_n), 'wb') as file:
                    pickle.dump(gmm, file)

            # Classify full dataset            
            print('Classifying full dataset into {} classes'.format(n_classes))
            data_classes = flt.gmm_classify(data_trans, gmm)
            data_probs = flt.gmm_prob(data_trans, gmm)
            print('Classification complete, writing to file'.format(n_classes))
            flt.write_tonc(data_classes.reset_index('n'),n_classes,m_id,'class',path_n)
            flt.write_tonc(data_probs.reset_index('n').mean('time'),n_classes,m_id,'probs',path_n)
            if os.path.isfile('{}/avg_prof.obj'.format(path_n)):
                continue
            else:
                # Calculate average profiles for each class
                avg_prof = flt.avg_profiles(data, data_classes, n_classes)
                print('Average profiles calculated, writing to file'.format(n_classes))
                with open('{}/avg_prof.obj'.format(path_n), 'wb') as file:
                    pickle.dump(avg_prof, file)      
            print('Done with {} classes'.format(n_classes))      
    
print('Done!')

Starting r1i1p1f2
No models found, generating training set
